<img src="https://docs.google.com/uc?id=1bbeZmwDKLQi476oJdTo5btPwGvfkCe8Q"/>


<br>
<center><h1>Solution proposal</h1></center>

The purpose of this notebook is to create a functional pipeline (data obtaining, data cleaning e feature engineering, until model training e score)


# 1. Solution strategy
    -> Project purpose
        -- 1. Which houses the company should be purchase?
        -- 2. With the house in company possession, what is the best moment to be sold, and for which price?
        -- 3. The company should do a restoration to increase the price sale? What are the recommendations? For each 
        restoration recommendation, how the price increases?

    -> Solution proposal
        -- 1. Table (House | Recommendation score) by EDA
            -- Houses that are cheap with high indicators of future appreciation
        -- 2. Table (House | Sale date | Sale price) --> ML Model
        -- 3. Table (House | Improvements Features | Sale price correlation/increase)

In [1]:
import pickle
import numpy as np
import pandas as pd
from scipy.special import inv_boxcox1p, boxcox1p
from math import sqrt

pd.set_option("display.max_columns", None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Solution proposal

### 1. Which houses the CEO should be purchased ?

Among the issues raised for the purchase of property, the location, future market valuation, and other conditions of the property were taken into consideration, besides the time difference between the purchase and sale being as short as possible to reduce the effects of a property degradation.

Based on the analysis, it is possible to stipulate the cheapest and most expensive m² per region, with the Bellevue region showing benefit indications of purchase since it has one of the cheapest m² about the land space (which can build) and it has one of the most expensive living space, even though it is a place with a high standard (houses above 1M on average) the purchase with a future market valuation or a renovate has indications of profit.

About sectors that have an annual appreciation, Seattle is one of the highest. In this case, the purchase recommendation occurs for the South Zone, which has an appreciation close to the North sector, but with cheaper houses (indicating that there is inflation in the North sector). Besides these sectors, the next one is Bellevue, reinforcing the above argument about buying for return through an improvement. In the less valued zone per year (Preston), purchases are not recommended to annual appreciation, since it has a high price average. The difference in valuation between the South Seattle sector, with cheaper houses, may present a better deal since it is possible to achieve the same average values considering the valuation market, but with a much smaller investment.

In terms of the conditions of the property, the ones that present better deals to purchase are those with conditions 2 and 3, higher than this, the increase in price sale is not significant. In condition 3 are high price variance, so it is possible to sell houses in these conditions for an extended kind of customers. And condition 2 shows a significant drop in price sale and can offer a better deal to improvements since it has a concentration of price, therefore improving the condition will cover more customer profiles.

Besides, the view of houses is relevant for customers, since views with waterfront are mostly overvaluing the house.

In [2]:
pd.read_csv('./data/recommended_houses.csv').head(15)

,id,price,bedrooms,Bathrooms,Sqft living,Floors,Waterfront,View,Condition,Grade,lat,long,neighborhood_name,Recommendation
0,2424059170,900000.00,5,6.00,7120,2.00,0,4,3,12,47.55,-122.11,"Mercer Island, Renton & Bellevue South",8.45
1,2626069030,1940000.00,4,6.00,7220,2.00,0,4,3,12,47.71,-122.01,Union Hill & Sammamish,8.43
2,6169901185,490000.00,5,4.00,4460,3.00,0,2,3,10,47.63,-122.37,Seattle North,8.27
3,3613600150,300523.00,3,2.00,2370,2.00,0,0,3,9,47.65,-122.37,Seattle North,8.13
4,2734100734,216650.00,3,4.00,1540,2.00,0,0,3,7,47.54,-122.32,Seattle South,8.06
5,2734100732,216650.00,3,4.00,1480,2.00,0,0,3,7,47.54,-122.32,Seattle South,8.05
6,7942601475,345600.00,5,4.00,2800,2.50,0,0,3,9,47.61,-122.31,Seattle South,8.00
7,5016003230,169317.00,2,1.00,790,1.00,0,2,3,7,47.62,-122.30,Seattle North,7.96
8,1832100030,597326.00,4,4.00,3570,2.00,0,0,3,10,47.58,-122.23,"Mercer Island, Renton & Bellevue South",7.92
9,922059169,800000.00,6,4.00,5480,2.00,0,0,4,10,47.41,-122.17,Kent & North Auburn,7.89


### 2. With the house in company possession, what is the best moment to be sold, and for which price?

The data show us that the best selling period occurs in spring and winter, where there is also an increase in demand. In this period, houses can be sold on average for an amount above 3% per m². About a period in which demand is low,  the difference between these two periods can be on average 6% per m².

In [3]:
model_file = './ml-models/XGB-final-log-target.pkl'
with open(model_file, 'rb') as f:
    model = pickle.load(f)
    
model

XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.87, gamma=0.03, gpu_id=-1,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.0173, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=937, n_jobs=-1, nthread=-1, num_parallel_tree=1,
             random_state=7, reg_alpha=0.9, reg_lambda=0.1, scale_pos_weight=1,
             subsample=0.73, tree_method=None, validate_parameters=False,
             verbosity=None)

In [4]:
data = pd.read_csv('./data/data-model-input.csv')
data.head()

,id,date,price,bedrooms,bathrooms,floors,view,condition,grade,sqft_above,sqft_basement,lat,long,total_rooms,year,part-date,yr_built_yr_purchase_dif,neighborhood_name,waterfront_0,waterfront_1,neighborhood_0,neighborhood_1,neighborhood_2,neighborhood_3,neighborhood_4,neighborhood_5,neighborhood_6,neighborhood_7,neighborhood_8,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,basement_living_0,basement_living_1,renovated_0,renovated_1,avg_zipcode_price,std_zipcode_price,min_zipcode_price,max_zipcode_price
0,7129300520,2014-10-13,12.31,9,1.00,1.00,0,3,49,12.60,0.00,47.51,-122.26,4.00,2014,2014-10,-59,Seattle South,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,12.50,0.40,12.08,12.98
1,6414100192,2014-12-09,13.20,9,4.00,2.00,0,3,49,14.44,9.72,47.72,-122.32,5.00,2014,2014-12,-63,Seattle North,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,12.96,0.32,12.63,13.41
2,5631500400,2015-02-25,12.10,4,1.00,1.00,0,3,36,11.40,0.00,47.74,-122.23,3.00,2015,2015-02,-82,"Redmond, Woodinville & Bellevue North",1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,12.99,0.31,12.61,13.32
3,2487200875,2014-12-09,13.31,16,9.00,1.00,0,5,49,12.26,11.86,47.52,-122.39,7.00,2014,2014-12,-49,Seattle South,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,13.09,0.38,12.75,13.64
4,1954400510,2015-02-18,13.14,9,4.00,1.00,0,3,64,13.64,0.00,47.62,-122.05,5.00,2015,2015-02,-28,Union Hill & Sammamish,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,13.37,0.30,13.01,13.71


In [5]:
lamb = 0.15
ids = data[['id', 'lat','long', 'neighborhood_name', 'date', 'year', 'part-date']]
X = data.drop(['id', 'price', 'lat', 'long', 'neighborhood_name', 'date', 'year', 'part-date'], axis = 1)
y = data['price']

In [6]:
for month in range(1, 13):
    X[f'month_{month}'] = 0

In [7]:
def sale_month(row):
    v = 0
    r = row.copy()
    for month in range(1, 13):
        row[f'month_{month}'] = 1
        predict = round(np.expm1(model.predict(row.values.reshape(1, -1))[0]))
        if predict > v:
            v = predict
            r['recommended_month_sale'] = month
            r['recommended_sale_price'] = v
            r[f'month_{month}'] = 1
            if month > 1:
                r[f'month_{m}'] = 0
            m = month
        row[f'month_{month}'] = 0
    return r

In [8]:
sale_month_predict = X.apply(sale_month, axis = 1)

### 3. The company should do a restoration to increase the price sale? What are the recommendations? For each restoration recommendation, how the price increases?

The improvement is interesting to increase the sale price, especially in houses that can be purchased with waterfront, since in these cases there is an overvaluation. In our analysis, it was possible to verify that in front of rivers and lakes, some houses were sold for relatively low prices, even though they have a quality of view 4 and are in sectors that are valued, such as Seattle. Among the attributes that can be classified for a restoration, such as the number of bathrooms, bedrooms, improvement in the condition, and the grade, the attribute that contributes the most is the grade, followed by the condition.

In [9]:
# improvements = ['bedrooms', 'bathrooms', 'condition', 'grade']

def improvement_increase(row, imp):
    if imp == 'bedrooms':
        # bedrooms -> sqft increment: 14m² --> 4 x 3.5m   
        v = sqrt(row[imp]) + 1
        row[imp] = pow(v, 2)
        
        rev = inv_boxcox1p(row['sqft_above'], lamb) + 14
        row['sqft_above'] = boxcox1p(rev, lamb)
        row['total_rooms'] += 1

    elif imp == 'bathrooms':
        # bathrooms -> sqft increment: 4.5m² --> 3 x 1.50m
        v = sqrt(row[imp]) + 1
        row[imp] = pow(v, 2)
        
        rev = inv_boxcox1p(row['sqft_above'], lamb) + 4.5
        row['sqft_above'] = boxcox1p(rev, lamb)
        row['total_rooms'] += 1
    
    elif imp == 'grade':
        v = sqrt(row['grade']) + 1
        v = 13 if v >= 13 else v
        row['grade'] = pow(v, 2)
        
    elif imp == 'condition':
        if row[imp] < 5:
            row[imp] += 1
        
    return round(np.expm1(model.predict(row.values.reshape(1, -1))[0]))  
    # sqft_above
    # total_rooms
    # sqft_living/total_rooms

In [10]:
bedroom_improv = sale_month_predict.drop(['recommended_month_sale', 'recommended_sale_price'], axis = 1).apply(improvement_increase, args=('bedrooms',), axis = 1)
bathroom_improv = sale_month_predict.drop(['recommended_month_sale', 'recommended_sale_price'], axis = 1).apply(improvement_increase, args=('bathrooms',), axis = 1)
condition_improv = sale_month_predict.drop(['recommended_month_sale', 'recommended_sale_price'], axis = 1).apply(improvement_increase, args=('condition',), axis = 1)
grade_improv = sale_month_predict.drop(['recommended_month_sale', 'recommended_sale_price'], axis = 1).apply(improvement_increase, args=('grade',), axis = 1)

In [11]:
sale_month_predict['price_increase_per_bedroom'] = bedroom_improv
sale_month_predict['price_increase_per_bathroom'] = bathroom_improv
sale_month_predict['price_increase_per_condition'] = condition_improv
sale_month_predict['price_increase_per_grade'] = grade_improv

In [12]:
def season_of_the_year(month):
    if month <= 3:
        return 'Winter'
    if month >= 4 and month <= 6:
        return 'Spring'
    if month >= 7 and month <= 9:
        return 'Summer'
    if month >= 10 and month <= 12:
        return 'Autumn'
        
sale_month_predict['season'] = sale_month_predict['recommended_month_sale'].apply(season_of_the_year)

In [13]:
export_data = ids.join([y, sale_month_predict])

In [14]:
export_data['price_increase_per_bedroom_perc'] = round(((bedroom_improv - export_data['recommended_sale_price'])/export_data['recommended_sale_price'])*100, 2)
export_data['price_increase_per_bathroom_perc'] = round(((bathroom_improv - export_data['recommended_sale_price'])/export_data['recommended_sale_price'])*100, 2)
export_data['price_increase_per_condition_perc'] = round(((condition_improv - export_data['recommended_sale_price'])/export_data['recommended_sale_price'])*100, 2)
export_data['price_increase_per_grade_perc'] = round(((grade_improv - export_data['recommended_sale_price'])/export_data['recommended_sale_price'])*100, 2)

In [15]:
export_data['price'] = np.expm1(export_data['price'])

In [16]:
export_data = export_data[['id', 'date','year', 'lat', 'long', 'neighborhood_name', 'price', 'bedrooms', 'bathrooms', 
                           'floors', 'view', 'condition', 'grade', 'recommended_month_sale', 'season', 
                           'recommended_sale_price', 'price_increase_per_bedroom', 'price_increase_per_bathroom', 
                           'price_increase_per_condition', 'price_increase_per_grade', 'part-date', 'price_increase_per_bedroom_perc',
                           'price_increase_per_bathroom_perc', 'price_increase_per_condition_perc', 'price_increase_per_grade_perc'
                          ]]

export_data['coordinates'] = export_data[['lat','long']].astype(str).apply(','.join,1)

In [17]:
export_data.to_csv('./data/data-model-output.csv', index=False)